# Transform: Hourly Summary Table of NYC Taxi Trips - Data Transformation

This notebook aggregates raw NYC Yellow Taxi data (January 2023 - March 2023) into a structured hourly summary table, which can be used for visualization and downstream analytics. DuckDB is used for efficient SQL-based transformation.

In [27]:
# Imports and configuration
import pandas as pd
import duckdb
from pathlib import Path
from glob import glob

# Paths
PROJECT_ROOT = Path("..")
RAW_DATA_PATHS = sorted(glob(str(PROJECT_ROOT / "airflow_pipeline" / "data" / "raw" / "yellow_tripdata_2023-0[1-3].parquet")))

OUTPUT_PATH = PROJECT_ROOT / "airflow_pipeline" / "data" / "processed" / "trip_summary_hourly.csv"

In [28]:
# Connect to DuckDB and register the Parquet file as a virtual table
con = duckdb.connect(database=":memory:")
con.execute(f"CREATE VIEW yellow_taxi AS SELECT * FROM read_parquet({RAW_DATA_PATHS})")

### Load and Execute SQL Query

The transformation logic is stored in a separate `.sql` file for better readability and maintainability.

### SQL Transformation

The query is stored externally in a `.sql` file for better maintainability. It generates a summary table with hourly-level metrics: trip count, average distance, total amount, and duration (in minutes).

In [29]:
# Read SQL query from .sql file, query is loaded from external file for clarity
query_path = PROJECT_ROOT / "airflow_pipeline" / "sql" / "raw" / "trip_summary_hourly.sql"
with open(query_path) as f:
    query = f.read()

# Execute query and load results
df_summary = con.execute(query).fetchdf()
df_summary.head()

ParserException: Parser Error: syntax error at or near "AVG"

### Export

The resulting summary is saved as a CSV file under the processed data folder.

In [ ]:
# Output the SQL querying results to csv files
df_summary.to_csv(OUTPUT_PATH, index=False)

print("Summary table generated and saved:", OUTPUT_PATH.resolve())

Summary table generated and saved: D:\DataTribes-Four-Week-Learning\data-engineering-learnings\test222\tuo-yang-nyc-taxi\airflow_pipeline\data\processed\trip_summary_hourly.csv


### Output Validation

Basic checks to ensure the summary table has the expected structure and no missing values.

In [ ]:
# Sanity checks
expected_rows = 24 * 90
actual_rows = len(df_summary)
null_count = df_summary.isnull().sum().sum()

print(f"Row check: {actual_rows} / {expected_rows}")
print(f"Missing values: {null_count}")

assert actual_rows == expected_rows # Excepting 2160 rows (hourly data for 90 days)
assert null_count == 0              # No missing values
print("Output shape and null check passed.")

Row check: 2160 / 2160
Missing values: 0
Output shape and null check passed.


### Summary

Hourly-level statistics have been successfully computed and saved. The output summary table can be used as a backend for visualizing key metrics such as trip volume, distance, and fare patterns over time.

Next step: build an interactive interface or dashboard to explore these trends more effectively — either within a web app (e.g., Streamlit) or using BI tools like Looker Studio.